In [1]:
import numpy as np
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import heapq
import math

In [2]:
devD_df = pd.read_csv('dev/devD.tsv', delimiter='\t', header=None)
trainD_df = pd.read_csv('train/trainD.tsv', delimiter='\t', header=None)

# applying groupby() function to 
# group the data on team value. 
tD_gk = trainD_df.groupby([0]) 
  
# Let's print the first entries 
# in all the groups formed. 
tD_gk.first()

trainD_df = tD_gk.aggregate(lambda x: tuple(x))

mlb = MultiLabelBinarizer()

trainD_enc = pd.DataFrame(mlb.fit_transform(trainD_df[1]),
                   columns= mlb.classes_, 
                   index= trainD_df.index).max(level=0)

trainD_enc['file'] = ""
for i in range(trainD_enc.shape[0]):
    f = open("train/text_files_en/"+ trainD_enc.index[i] + ".txt", "r")
    trainD_enc['file'][i] = f.read()
    
# applying groupby() function to 
# group the data on team value. 
tD_gk = devD_df.groupby([0]) 
  
# Let's print the first entries 
# in all the groups formed. 
tD_gk.first()

devD_df = tD_gk.aggregate(lambda x: tuple(x))

mlb = MultiLabelBinarizer()
devD_enc = pd.DataFrame(mlb.fit_transform(devD_df[1]),
                   columns= mlb.classes_, 
                   index= devD_df.index).max(level=0)

devD_enc['file'] = ""
for i in range(devD_enc.shape[0]):
    f = open("dev/text_files_en/"+ devD_enc.index[i] + ".txt", "r")
    devD_enc['file'][i] = f.read()
    
columns_train = trainD_enc.columns
columns_dev = devD_enc.columns

count = 0
for x in columns_dev:
    if x not in columns_train:
        trainD_enc[x] = 0
        
count = 0
for x in columns_train:
    if x not in columns_dev:
        devD_enc[x] = 0 
        
train = trainD_enc['file']
train = pd.DataFrame(train)

del trainD_enc['file']

train['tar'] = trainD_enc.values.tolist()

dev = devD_enc['file']
dev = pd.DataFrame(dev)

del devD_enc['file']

dev['tar'] = devD_enc.values.tolist()

train = pd.concat([train, dev.iloc[:200]])

dev = dev.iloc[200:]

train = train.reset_index()
#dev = dev.reset_index()

del train[0]
#del dev[0]

train = train.rename(columns={"file": "text", "tar": "labels"})
dev = dev.rename(columns={"file": "text", "tar": "labels", 0:'index'})

/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [4]:
train['text'] = train['text'].apply(lambda x: remove_punctuation(x), 1)
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
train['text'] = train['text'].apply(lambda x: remove_stopwords(x))
train['text'] = train['text'].apply(lambda x: word_lemmatizer(x))
#train['text'] = train['text'].apply(lambda x: word_stemmer(x))

In [5]:
def split_sentences(df, leng, overlap):               # leng is excluding the overlap
    new_df = pd.DataFrame(columns=df.columns)         # new df with same coulmns
    row = 0
    for i in range(df.shape[0]):                      # loop through df rows
        words = df['text'].loc[i].split()
        count = 0
        for j in range(math.ceil(len(words)/leng)):   #loop through sentence
            new_df.loc[row] = df.loc[i]
            new_df['text'].loc[row] = ""
            for k in range (leng + overlap):
                if(j*leng+k < len(words)):
                    new_df['text'].loc[row] += words[j*leng + k] + " "
            row += 1
    return new_df

In [6]:
temp = split_sentences(train, 100, 10)
temp

,text,labels
0,describe case 37yearold man previous active li...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,neurological examination revealed meningeal si...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,transthoracic echocardiography show significan...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,report case 29yearold woman underwent pelvic u...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,cytoplasm well nucleus slightly eosinophilic m...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1663,blood vitreous humor pericardial fluid pleural...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
1664,n meningitidis serogroup c watersenhouse syndr...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
1665,73yearold woman came emergency department acci...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1666,condition joint stable varus valgus extension ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
models = [['bert', 'bert-base-uncased'], ['roberta', 'roberta-base-uncased'], ['bert', 'emilyalsentzer/Bio_ClinicalBERT']]

In [7]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel(
    "bert", # model name (for bert change to "bert")
    "emilyalsentzer/Bio_ClinicalBERT", # pretrained model (for bert change to "bert-base-uncased")
    num_labels= 2194,
    args={"num_train_epochs": 10, "train_batch_size": 5, "fp16_opt_level" : 'O0',
    "gradient_accumulation_steps": 2, "overwrite_output_dir": True,  "learning_rate": 4e-5, "max_seq_length": 128, "save_eval_checkpoints": False, "save_model_every_epoch": False},
)

In [8]:
model.train_model(temp)

Converting to features started. Cache is not used.



Selected optimization level O0:  Pure FP32 training.

Defaults for this optimization level are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0


Running loss: 0.092958


Running loss: 0.038578


Running loss: 0.033976


...

Running loss: 0.020658


Running loss: 0.038576


Running loss: 0.045130


Running loss: 0.034724


Running loss: 0.027510

Training of bert model complete. Saved to outputs/.


In [9]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(dev, acc=sklearn.metrics.label_ranking_loss)
print(result)
print(model_outputs)


{'LRAP': 0.02148876276917489, 'acc': 0.3250209402326321, 'eval_loss': 0.030885874426790645}
[[0.00842041 0.00679542 0.00743072 ... 0.00236768 0.00243979 0.00239703]
 [0.00841848 0.00679426 0.00742733 ... 0.00236641 0.00243892 0.00239605]
 [0.00842116 0.00679574 0.00742968 ... 0.00236763 0.00243991 0.00239653]
 ...
 [0.00841997 0.0067957  0.00742833 ... 0.0023671  0.00243934 0.00239683]
 [0.00842177 0.00679657 0.00742897 ... 0.00236743 0.00243921 0.00239699]
 [0.00841818 0.00679567 0.00742887 ... 0.00236632 0.00243866 0.00239714]]
